In [278]:
def members_committes(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    rows = soup.findAll('tr')
    num = []
    for i in range(0,len(rows)):
        if any(re.findall('PRESIDENCIA|SECRETARÍA|INTEGRANTES', rows[i].text)):
            num.append(i)
            
    rows = soup.findAll('tr')[num[0] + 1:]
    temp = {}
    for i in rows:
        if len(i.findAll('td')) == 1:
            posicion = i.findAll('td')[0].text
            temp[posicion] = []
        else:
            integrante = re.sub('\n+', '', i.findAll('td')[0].text)
            clean_integrante = re.sub('[ ]+', ' ', integrante)
            clean_integrante = re.sub('^[ ]+', '', clean_integrante)
            clean_integrante = re.sub('[ ]+$', '', clean_integrante)
            temp[posicion].append(clean_integrante)
    
    data_members = {}
    try:
        for x in temp['PRESIDENCIA']:
            data_members[x] = 'PRESIDENCIA'
    except:
        print(link)
    try:
        for x in temp['SECRETARÍA']:
            data_members[x] = 'SECRETARIA'
    except:
        print(link)

    for x in temp['INTEGRANTES']:
        data_members[x] = 'INTEGRANTES'
    return(data_members)

In [217]:
import requests
import re
import json
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup

In [305]:
links = ['http://sitl.diputados.gob.mx/LXIV_leg/listado_de_comisioneslxiv.php?tct=1',
         #'http://sitl.diputados.gob.mx/LXIV_leg/listado_de_comisioneslxiv.php?tct=2',
        'http://sitllxiii.diputados.gob.mx/listado_de_comisioneslxiii.php?tct=1', 
         'http://sitllxiii.diputados.gob.mx/listado_de_comisioneslxiii.php?tct=2', 
        'http://sitllxii.diputados.gob.mx/listado_de_comisioneslxii.php?tct=1', 
         'http://sitllxii.diputados.gob.mx/listado_de_comisioneslxii.php?tct=2', 
        'http://sitllxi.diputados.gob.mx/listado_de_comisioneslxi.php?tct=1', 
         'http://sitllxi.diputados.gob.mx/listado_de_comisioneslxi.php?tct=2', 
        'http://sitllx.diputados.gob.mx/listado_de_comisiones.php?tct=1', 
        'http://sitllx.diputados.gob.mx/listado_de_comisiones.php?tct=2']

In [306]:
r = requests.get(links[5])
soup = BeautifulSoup(r.content, 'html.parser')

In [307]:
link_integrantes = []
name_comisiones = []
for i in soup.findAll('a'):
    link = i.get('href')
    if any(re.findall('integrantes', link)):
        #link = re.sub('^[.]', '', link) #63 legislatura
        link_integrantes.append('http://sitllxi.diputados.gob.mx/' +link)
        name_comisiones.append(re.sub('\n', '', i.text))
    else:
        continue

In [308]:
list_members = []
list_committes = []
name_comisiones2 = []
for i, e in tqdm(zip(link_integrantes, name_comisiones), total = len(link_integrantes)):
    try:
        dict_ = members_committes(i)
        for x, y in dict_.items():
            list_members.append(x)
            list_committes.append(y)
            name_comisiones2.append(e)
    except:
        print(i, 'error')

100%|██████████| 44/44 [00:10<00:00,  4.10it/s]


In [309]:
final_dict = []
for x, y, z in zip(list_members, list_committes, name_comisiones2):
    temp = {}
    temp['legislatura'] = 61
    temp['name_inc'] = x
    temp['position_committe'] = y
    temp['name_comision'] = z
    temp['type_committe'] = 'especial'
    final_dict.append(temp)

In [310]:
pd.DataFrame(final_dict)

,legislatura,name_inc,position_committe,name_comision,type_committe
0,61,López Aguilar Cruz,PRESIDENCIA,Agricultura y Ganadería,especial
1,61,Lugo Oñate Alfredo Francisco,SECRETARIA,Agricultura y Ganadería,especial
2,61,Montes Cavazos Fermín,SECRETARIA,Agricultura y Ganadería,especial
3,61,Zubia Rivera Rolando,SECRETARIA,Agricultura y Ganadería,especial
4,61,Torres Santos Sergio Arturo,SECRETARIA,Agricultura y Ganadería,especial
...,...,...,...,...,...
1139,61,Ovando Patrón José Luis,INTEGRANTES,Vivienda,especial
1140,61,Peralta Rivas Pedro,INTEGRANTES,Vivienda,especial
1141,61,Puppo Gastélum Silvia,INTEGRANTES,Vivienda,especial
1142,61,Serrano Hernández Maricela (LICENCIA),INTEGRANTES,Vivienda,especial


In [311]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/01-collection_data/05-committe_activity/members_committe61_especial.json', 'w') as d:
    json.dump(final_dict, d)